In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('../../')

In [3]:
%cd /content
!git clone https://github.com/abbkevkla/MetalSongGenerator

%cd /content/MetalSongGenerator/musicautobot-master

/content
Cloning into 'MetalSongGenerator'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 287 (delta 33), reused 48 (delta 16), pack-reused 184
Receiving objects: 100% (287/287), 6.42 MiB | 8.50 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/MetalSongGenerator/musicautobot-master


In [0]:
from music21 import *

In [5]:
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
# Strunta i varningen om MuseScore, det behövs inte för att träna

## MusicTransformer Training

MusicTransformer takes the basic idea of [Language Models](https://en.wikipedia.org/wiki/Language_model) and applies it to Music.  

Given a sequence of notes, predict the next most likely set of notes.

This model is based off of [transformer-XL](https://arxiv.org/abs/1901.02860) and uses fast.ai's [implementation](https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py) of it.

In [0]:
# Location of your midi filesfiles
midi_path = Path('data/midi/Solos') # At least 10 midi files needed
midi_path.mkdir(parents=True, exist_ok=True)

# Location to save dataset
data_path = Path('data/numpy')
data_path.mkdir(parents=True, exist_ok=True)

data_save_name = 'musicitem_data_save.pkl'

## 1. Gather midi dataset

Make sure all your midi data is in `musicautobot/data/midi` directory

Here's a pretty good dataset with lots of midi data:  
https://www.reddit.com/r/datasets/comments/3akhxy/the_largest_midi_collection_on_the_internet/

Download the folder and unzip it to `data/midi`

## 2. Create dataset from MIDI files

In [7]:
midi_files = get_files(midi_path, '.mid', recurse=True); len(midi_files)

10

In [0]:
processors = [Midi2ItemProcessor()]
data = MusicDataBunch.from_files(midi_files, data_path, processors=processors, bs=2, bptt=12)
data.save(data_save_name)

In [9]:
# Show Data
data.train_dl.on_epoch_begin()
x, y = data.one_batch();
x, y

(tensor([[[  0,   0],
          [  1,   0],
          [ 66,   0],
          [145,   0],
          [ 59,   0],
          [138,   0],
          [ 59,   0],
          [145,   0],
          [  8,   0],
          [138,   0],
          [ 62,   1],
          [138,   1]],
 
         [[138,  52],
          [ 78,  53],
          [138,  53],
          [  8,  53],
          [138,  53],
          [ 77,  54],
          [138,  54],
          [  8,  54],
          [138,  54],
          [ 76,  55],
          [138,  55],
          [  8,  55]]]), tensor([[[  1,   0],
          [ 66,   0],
          [145,   0],
          [ 59,   0],
          [138,   0],
          [ 59,   0],
          [145,   0],
          [  8,   0],
          [138,   0],
          [ 62,   1],
          [138,   1],
          [  8,   1]],
 
         [[ 78,  53],
          [138,  53],
          [  8,  53],
          [138,  53],
          [ 77,  54],
          [138,  54],
          [  8,  54],
          [138,  54],
          [ 76,  55],
  

## 3. Load Model

In [0]:
batch_size = 1
encode_position = True
dl_tfms = [batch_position_tfm] if encode_position else []
data = load_data(data_path, data_save_name, bs=batch_size, encode_position=encode_position, dl_tfms=dl_tfms)

In [0]:
config = default_config()
config['encode_position'] = encode_position
learn = music_model_learner(data, config=config.copy())

## 4. Train

In [12]:
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,2.658623,4.030755,0.302597,00:25
1,2.738264,3.470382,0.302597,00:25
2,2.917616,3.351574,0.302597,00:25


In [0]:
learn.save('solos_model')

PosixPath('data/numpy/models/solos_model.pth')